In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import seaborn as sns
import pandas as pd

from pasna_analysis import Experiment, Embryo, Trace, utils

experiment_name = '20240515'
exp_path = Path.cwd().parent.joinpath('data',experiment_name)
exp = Experiment(exp_path)

In [ ]:
'''Removes all embryos where the first peak was detected before 30
minutes.'''

FIRST_PEAK_THRESHOLD = 30*60
activities = exp.activities()
lengths = exp.lengths()

embryos = [Embryo(a,l) for a, l in zip(activities, lengths)]
traces = {e.name: None for e in embryos}

to_remove = []
for emb in embryos:
    time, act, stct = emb.activity[:, 0], emb.activity[:, 1], emb.activity[:, 2], 
    trace = Trace(time, act, stct)
    if trace.get_first_peak_time() < FIRST_PEAK_THRESHOLD:
        print(f'First peak detected before 30 mins of imaging for {emb.name} (t={trace.get_first_peak_time()/60} mins). Skipping..')
        to_remove.append(emb.name)
    else:
        traces[emb.name] = trace

embryos = [e for e in embryos if e.name not in to_remove]

In [ ]:
'''List of all metrics calculated for a dff trace:'''
i = 0

emb = embryos[i]
trace = traces[emb.name]

print('Peak times (secs):')
print(trace.peak_times)

print('Peak amplitudes:')
print(trace.peak_amplitudes)

print('Peak intervals (secs):')
print(trace.peak_intervals)

print('Trace RMS:')
print(trace.rms)

print('Indices of each peak (start and end):')
print(trace.peak_bounds_indices)

print('Indices of each peak:')
print(trace.peak_idxes)

print('Duration of each peak (s):')
print(trace.peak_durations)

print('Rise time of each peak (s):')
print(trace.peak_rise_times)

print('Decay time of each peak (s):')
print(trace.peak_decay_times)

print('Area under the curve for each episode:')
print(trace.peak_aucs)

In [ ]:
amplitudes = {'amp': [], 'peak_num': [], 'name': []}
for emb in embryos:
    if traces[emb.name] is None:
        continue 
    trace = traces[emb.name]
    for i, amp in enumerate(trace.peak_amplitudes):
        amplitudes['peak_num'].append(i)
        amplitudes['amp'].append(amp)
        amplitudes['name'].append(emb.name)

amps = pd.DataFrame(amplitudes)
sns.set_theme(style='darkgrid')
sns.lineplot(data=amps, x='peak_num', y='amp', errorbar='sd');
# sns.lineplot(data=amps, x='peak_num', y='amp', hue='name');